# eICU data - dataset construction intakeoutput

In [8]:
import pandas as pd

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 300)

In [9]:
patients = pd.read_csv('../2.0/patient.csv')
inout = pd.read_csv('../2.0/intakeOutput.csv.gz')
patients_inout = patients.merge(inout, how='inner', on='patientunitstayid')

In [19]:
patients_inout = patients_inout[patients_inout['intakeoutputoffset'] >=0 ]

#51 Variables overlapping between eICU data ,intakeoutput table, and MIMIC III data ,inputevent_cv and outputevent_cv tables.
subset = ['Voided Amount','chest tubes','Gastric Tube Intake','Emesis','Enteral Tube Intake: Nasogastric Nostril, L','Stool',
          'Wound/Drain #1','URINE CATHETER','Tube Feeding Residual Discarded (mL)','Wound/Drain #2','Wound/Drain #1'
          'Surgical Drain, Tube Output: Bulb-closed system Le','Surgical Drain, Tube Output: Bulb-closed system Th',
          'Drain 1 Output mL','Surgical Drain, Tube Output: Bulb-closed system Ab','Surgical Drain, Tube Output: Bulb-closed system Ne',
          'Banana bag','chest tube meds','chest tube pleurals','Chest Tubes Meds','chest tubes meds',
          'Albumin 5%','Volume (mL)-dextrose 5 % / sodium chloride 0.4...','Lorazepam Volume','Volume (mL)-calcium GLUCONATE IVPB premix 2 g',
          'Volume (ml) Midazolam','Phenylephrine','Volume (mL)-furosemide (LASIX) injection 40 mg','Volume (mL) Norepinephrine',
          'Volume (ml) Magnesium Sulfate','Nitroglycerin','Volume (ml) Insulin','insulin regular','Volume (mL)-potassium chloride IVPB 20 mEq 100 mL',
          'Volume-Transfuse red blood cells','D5 1/2NS','LR','k phos','Volume (ml)  Metoprolol','PO intake',
           'Wound/Drain #2','Wound/Drain #1','Drain 1 Output mL','Surgical Drain, Tube Output: Bulb-closed system Ab',
            'Surgical Drain, Tube Output: Bulb-closed system Ne','insulin','INSULIN','Insulin drip','Insulin',
            'INSULIN GTT','INSULIN GTT','Insulin Drip']


patients_inout = patients_inout.loc[patients_inout['celllabel'].isin(subset)]
patients_inout

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,hospitaladmitoffset,hospitaladmitsource,hospitaldischargeyear,hospitaldischargetime24,hospitaldischargeoffset,hospitaldischargelocation,hospitaldischargestatus,unittype,unitadmittime24,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid,intakeoutputid,intakeoutputoffset,intaketotal,outputtotal,dialysistotal,nettotal,intakeoutputentryoffset,cellpath,celllabel,cellvaluenumeric,cellvaluetext
7,141168,128919,Female,70,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",152.4,15:54:00,0,Direct Admit,2015,03:50:00,3596,Death,Expired,Med-Surg ICU,15:54:00,Direct Admit,1,admit,84.3,85.80,03:50:00,3596,Death,Expired,002-34851,13307419,2346,993.00,0.0,0.0,993.00,2346,flowsheet|Flowsheet Cell Labels|I&O|Intake (ml...,Volume (ml) Midazolam,7.00,7.00
8,141168,128919,Female,70,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",152.4,15:54:00,0,Direct Admit,2015,03:50:00,3596,Death,Expired,Med-Surg ICU,15:54:00,Direct Admit,1,admit,84.3,85.80,03:50:00,3596,Death,Expired,002-34851,13307420,2346,993.00,0.0,0.0,993.00,2346,flowsheet|Flowsheet Cell Labels|I&O|Intake (ml...,Volume (mL) Norepinephrine,317.00,317.00
62,141194,128941,Male,68,Caucasian,73,92,"Sepsis, renal/UTI (including bladder)",180.3,18:18:40,-780,Floor,2015,23:30:00,12492,Home,Alive,CTICU,07:18:00,Floor,1,admit,73.9,76.70,15:31:00,4813,Floor,Alive,002-5276,10977583,1722,2818.60,200.0,0.0,2618.60,1722,flowsheet|Flowsheet Cell Labels|I&O|Intake (ml...,Volume (mL) Norepinephrine,15.60,15.60
87,141194,128941,Male,68,Caucasian,73,92,"Sepsis, renal/UTI (including bladder)",180.3,18:18:40,-780,Floor,2015,23:30:00,12492,Home,Alive,CTICU,07:18:00,Floor,1,admit,73.9,76.70,15:31:00,4813,Floor,Alive,002-5276,12134246,2202,1858.22,825.0,0.0,1033.22,2202,flowsheet|Flowsheet Cell Labels|I&O|Intake (ml...,Volume (mL) Norepinephrine,4.70,4.70
151,141194,128941,Male,68,Caucasian,73,92,"Sepsis, renal/UTI (including bladder)",180.3,18:18:40,-780,Floor,2015,23:30:00,12492,Home,Alive,CTICU,07:18:00,Floor,1,admit,73.9,76.70,15:31:00,4813,Floor,Alive,002-5276,14749570,4242,0.00,650.0,0.0,-650.00,4242,flowsheet|Flowsheet Cell Labels|I&O|Output (ml...,Stool,300.00,300.00
352,141233,128973,Female,81,Caucasian,73,92,Mitral valve replacement,165.1,17:27:13,-1,Operating Room,2015,15:48:00,22940,Home,Alive,CTICU,17:28:00,Operating Room,1,admit,61.7,73.20,14:53:00,15685,Floor,Alive,002-8979,10446533,1532,209.90,160.0,0.0,49.90,1532,flowsheet|Flowsheet Cell Labels|I&O|Intake (ml...,Volume (ml) Insulin,2.00,2.00
380,141233,128973,Female,81,Caucasian,73,92,Mitral valve replacement,165.1,17:27:13,-1,Operating Room,2015,15:48:00,22940,Home,Alive,CTICU,17:28:00,Operating Room,1,admit,61.7,73.20,14:53:00,15685,Floor,Alive,002-8979,11606807,2912,335.00,0.0,0.0,335.00,2912,flowsheet|Flowsheet Cell Labels|I&O|Intake (ml...,Volume-Transfuse red blood cells,335.00,335.00
381,141233,128973,Female,81,Caucasian,73,92,Mitral valve replacement,165.1,17:27:13,-1,Operating Room,2015,15:48:00,22940,Home,Alive,CTICU,17:28:00,Operating Room,1,admit,61.7,73.20,14:53:00,15685,Floor,Alive,002-8979,11615845,14472,0.00,1.0,0.0,-1.00,14472,flowsheet|Flowsheet Cell Labels|I&O|Output (ml...,Stool,1.00,1.00
386,141233,128973,Female,81,Caucasian,73,92,Mitral valve replacement,165.1,17:27:13,-1,Operating Room,2015,15:48:00,22940,Home,Alive,CTICU,17:28:00,Operating Room,1,admit,61.7,73.20,14:53:00,15685,Floor,Alive,002-8979,11661599,3032,407.40,100.0,0.0,307.40,3032,flowsheet|Flowsheet Cell Labels|I&O|Intake (ml...,Volume (ml) Insulin,4.30,4.30
408,141233,128973,Female,81,Caucasian,73,92,Mitral valve replacement,165.1,17:27:13,-1,Operating Room,2015,15:48:00,22940,Home,Alive,CTICU,17:28:00,Operating Room,1,admit,61.7,73.20,14:53:00,15685,Floor,Alive,002-8979,11879073,57

### Cleaning

In [11]:
frequent_inouts = patients_inout.groupby('celllabel')['patientunitstayid'].nunique().sort_values(ascending=False)

long_inouts = patients_inout.groupby('celllabel').apply(lambda x: x.groupby('patientunitstayid').size().mean()).sort_values(ascending=False)
selected_inouts = set(list(long_inouts.index)) & set(list(frequent_inouts.index))
# To have a look at what lab measurements you have selected
inouts_selected_df = pd.concat([frequent_inouts.loc[selected_inouts], long_inouts.loc[selected_inouts]], axis=1).rename({'patientunitstayid': 'icu_stays', 0: 'avg_length'}, axis=1)
print(inouts_selected_df.to_string())

patients_inout_selected = patients_inout[patients_inout['celllabel'].isin(selected_inouts)]


                                                    icu_stays  avg_length
celllabel                                                                
Volume (ml)  Metoprolol                                    16   50.875000
Chest Tubes Meds                                            2   75.500000
PO intake                                                1285    8.386770
Volume (mL)-calcium GLUCONATE IVPB premix 2 g             227    1.753304
Stool                                                   39229    4.655153
Voided Amount                                            2121   15.087694
Volume (ml) Midazolam                                     689   17.193033
Surgical Drain, Tube Output: Bulb-closed system Ab        646    9.732198
Tube Feeding Residual Discarded (mL)                      921   16.780673
Surgical Drain, Tube Output: Bulb-closed system Th         15    8.933333
Emesis                                                   3500    2.385143
chest tubes meds                      

In [12]:
patients_inout_selected['inout_times'] = (patients_inout_selected['intakeoutputoffset'])
inout_dict=dict(zip(list(selected_inouts),range(len(selected_inouts))))
patients_inout_selected['cell_code']=patients_inout_selected['celllabel'].map(inout_dict)

In [14]:
patients_inout_selected_short = patients_inout_selected[['patientunitstayid', 'inout_times', 'cell_code','cellvaluenumeric']].rename({'patientunitstayid': 'id'}, axis=1)
patients_inout_selected_short_gb = patients_inout_selected_short.groupby(['id', 'inout_times', 'cell_code'], as_index=False).mean()

In [15]:
patients_inout_selected_short_gb

,id,inout_times,cell_code,cellvaluenumeric
0,141168,2346,6,7.00
1,141168,2346,18,317.00
2,141194,1722,18,15.60
3,141194,2202,18,4.70
4,141194,4242,4,300.00
5,141233,92,32,10.00
6,141233,305,25,199.00
7,141233,572,32,24.00
8,141233,1052,32,17.00
9,141233,1172,32,1.44


In [ ]:
patients_inout_selected_short_gb.to_csv('intakeoutput_processed.csv')